In [63]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec  4 18:35:51 2017

@author: Michael
"""

# import the required libraries
import folium
import pandas as pd
from geopy.geocoders import Nominatim
import requests
import json
 
# Create an empty map
worldMap = folium.Map(tiles='Mapbox Bright', location=[46.2222, 14.6073], zoom_start=5, min_zoom=4, max_zoom=10)

#dictionary that will be filled by API data 
apiData = {}
allData = []
cities = []
levels = []

#create colour array that corresponds to PM10 indices
colour = ["rgb(212,224,237)","rgb(15,112,1)","rgb(255,255,11)","rgb(222,44,35)","rgb(62,26,121)"]

#create level array that corresponds to the pollution index
levels = ["Very Low", "Low", "Moderate", "High", "Very High"]

#handle index range
def getIndex(item):
    annualMean = float(item)
    index = 1
    if(annualMean > 14 and annualMean < 21):
        index = 2
    if(annualMean > 21 and annualMean < 28):
        index = 3
    if(annualMean > 28 and annualMean < 35):
        index = 4
    if(annualMean > 35 and annualMean < 42):
        index = 5
    if(annualMean > 42 and annualMean < 50):
        index = 6
    if(annualMean > 50 and annualMean < 57):
        index = 7
    if(annualMean > 57 and annualMean < 71):
        index = 8
    if(annualMean > 71 and annualMean < 92):
        index = 9
    if(annualMean > 92):
        index = 10
    return index

#get level from index
def getLevel(index):
    level = 0
    if(index==2 or index==3):
        level=1
    if(index==4 or index==5 or index==6):
        level=2
    if(index==7 or index==8 or index==9):
        level=3
    if(index==10):
        level=4
    return level
        
    
def callAPI():
    with requests.session() as s:
        resp = s.get("http://www.deepbreath.ai/michael.php")
        values = resp.text
        valuesJson = json.loads(values)
        for item,data in valuesJson.items():
            if(data['city'] in cities):
               continue
            else:
                case = {'city': data['city'], 'longitude': data['longitude'], 'latitude': data['latitude'], 'pm10': data['pm10']}
                allData.append(case)
                cities.append(data['city'])
           
def sketchAPIData():
    callAPI()
    for data in allData:
        lon = data['longitude']
        lat = data['latitude']
        cityName = data['city']
        label = cityName + "  -  "
        if(data['pm10'] != None):
            airIndex = getIndex(data['pm10'])
            airLevel = getLevel(airIndex)
            airColor = colour[airLevel]
            label += str(data['pm10']) + " ug/m3" + " (" +str(levels[airLevel]) +")"
            folium.CircleMarker([float(lat), float(lon)],
                                radius=int(data['pm10']) / 3,
                                color=airColor,
                                popup= label,
                                fill=True,
                                fill_color=airColor,
                                line_color=airColor,
                                fill_opacity=0.15
                               ).add_to(worldMap)

def sketchLocalDataset():
    geolocator = Nominatim()
    #read csv file from local dir
    df = pd.read_csv('C:/Users/Michael/Desktop/trimmedCities.csv')
    df.head()
    for item in range(len(df)):
        lon = geolocator.geocode(df['City/Town'][item]).longitude
        lat = geolocator.geocode(df['City/Town'][item]).latitude
        airIndex = getIndex(df['Annual mean, ug/m3'][item])
        airColor = colour[airIndex]
        folium.CircleMarker([float(lat), float(lon)],
                                radius=int(data['pm10']) / 3,
                                color=airColor,
                                popup= label,
                                fill=True,
                                fill_color=airColor,
                                line_color=airColor,
                                fill_opacity=0.15
                               ).add_to(worldMap)

sketchAPIData()

#add legend to map
import branca.colormap as cm
colormap = cm.LinearColormap(
    ["#D4E0ED","#0F7001","#FFFF0B","#DE2C23", "#3E1A79"],
    vmin=1, vmax=11,
    index=[1,2,4,7,10]
    #index=["very low", "low", "moderate", "high", "very high"]
)
colormap.caption = 'Air index'
worldMap.add_child(colormap)

worldMap.save('pm10Map.html')
